# Análise Preditiva com Regressão Linear: Comparação entre Três Datasets

## 1. Introdução

Este trabalho apresenta uma análise comparativa do desempenho do algoritmo de Regressão Linear aplicado a três conjuntos de dados distintos relacionados a variáveis ambientais. O objetivo principal é investigar a relação entre diferentes pares de variáveis e avaliar a capacidade preditiva do modelo em cada cenário.

### 1.1 Datasets Utilizados

1. **Dataset 1: Umidade vs Temperatura**
   - Objetivo: Prever umidade relativa do ar com base na temperatura
   - Contexto: Dados atmosféricos com possível necessidade de conversão de unidades

2. **Dataset 2: Temperatura Mínima vs Temperatura Máxima**
   - Objetivo: Prever a temperatura máxima diária com base na temperatura mínima
   - Contexto: Dados meteorológicos sequenciais

3. **Dataset 3: Salinidade vs Temperatura da Água**
   - Objetivo: Prever temperatura da água com base na salinidade
   - Contexto: Dados oceanográficos

## 2. Metodologia

### 2.1 Abordagem Geral

O workflow implementado segue as etapas padrão de um projeto de machine learning:

```python
# Fluxo do pipeline implementado
1. Carregamento e inspeção dos dados
2. Pré-processamento e limpeza
3. Conversão de unidades (quando necessário)
4. Divisão treino-teste (70%-30%)
5. Treinamento do modelo de Regressão Linear
6. Avaliação com métricas robustas
7. Visualização dos resultados
```

### 2.2 Algoritmo Escolhido: Regressão Linear

**Justificativa da escolha:**
- Simplicidade e interpretabilidade
- Eficiência computacional
- Adequação para problemas de regressão com relações lineares
- Base sólida para comparações futuras com modelos mais complexos

### 2.3 Métricas de Avaliação

Foram utilizadas três métricas principais para avaliação dos modelos:

- **MAE (Mean Absolute Error)**: Erro absoluto médio → Facilidade de interpretação
- **MSE (Mean Squared Error)**: Erro quadrático médio → Penaliza erros grandes
- **R² (Coeficiente de Determinação)**: Proporção da variância explicada → Qualidade do ajuste

## 3. Implementação do Código

### 3.1 Importação de Bibliotecas

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

### 3.2 Funções Auxiliares

In [ ]:
def preprocess_temperature(df, temp_col, unit_col):
    """Converte temperaturas de Fahrenheit para Celsius quando necessário"""
    def convert(row):
        if unit_col and row.get(unit_col, "").lower() in ['f', 'fahrenheit']:
            return (row[temp_col] - 32) * 5/9
        return row[temp_col]
    df[temp_col] = df.apply(convert, axis=1)
    return df

def load_dataset(path, conversions=None):
    """Carrega dataset CSV e aplica conversões de temperatura"""
    df = pd.read_csv(path, low_memory=False)
    if conversions:
        for temp_col, unit_col in conversions:
            if unit_col and unit_col in df.columns:
                df = preprocess_temperature(df, temp_col, unit_col)
    return df

def train_and_evaluate(df, feature_col, target_col, dataset_name, save_path=None):
    """Executa pipeline completo de treinamento e avaliação do modelo"""
    # Remove linhas com valores missing
    df = df.dropna(subset=[feature_col, target_col])
    
    # Prepara features (X) e target (y)
    X = df[[feature_col]].values
    y = df[target_col].values

    # Divide os dados em treino (70%) e teste (30%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Cria e treina o modelo de regressão linear
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Faz previsões no conjunto de teste
    y_pred = model.predict(X_test)

    # Calcula métricas de avaliação
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Exibe resultados
    print(f"\n=== Resultados para {dataset_name} ===")
    print(f"MAE: {mae:.3f}")
    print(f"MSE: {mse:.3f}")
    print(f"R² : {r2:.3f}")

    # Cria gráfico de dispersão comparando valores reais vs previstos
    plt.figure(figsize=(8, 6))
    plt.scatter(y_test, y_pred, alpha=0.6)
    plt.xlabel("Valores Reais")
    plt.ylabel("Valores Previstos")
    plt.title(f"Reais vs Previstos - {dataset_name}")
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')

    # Salva o gráfico se um caminho for fornecido
    if save_path:
        plt.savefig(save_path, bbox_inches="tight")
    plt.show()

    return {"MAE": mae, "MSE": mse, "R2": r2}

### 3.3 Configuração de Diretórios

In [ ]:
DATA_PATH = "data"
RESULTS_PATH = os.path.join("results", "graficos")
os.makedirs(RESULTS_PATH, exist_ok=True)

## 4. Execução e Resultados

### 4.1 Dataset 1: Umidade vs Temperatura

In [ ]:
print("=== DATASET 1: Umidade vs Temperatura ===")
df1 = load_dataset(
    os.path.join(DATA_PATH, "ds_hum_vs_temp_dirty.csv"), 
    conversions=[("temperatura", "unidade_temperatura")]
)
print(f"Dimensões do dataset: {df1.shape}")
print(f"Colunas disponíveis: {list(df1.columns)}")

metrics1 = train_and_evaluate(
    df1, "temperatura", "humidade", "Umidade vs Temperatura", 
    save_path=os.path.join(RESULTS_PATH, "umidade_vs_temp.png")
)

### 4.2 Dataset 2: MinTemp vs MaxTemp

In [ ]:
print("=== DATASET 2: MinTemp vs MaxTemp ===")
df2 = load_dataset(
    os.path.join(DATA_PATH, "ds_min_temp_vs_max_temp_raw.csv"), 
    conversions=[("MinTemp", None), ("MaxTemp", None)]
)
print(f"Dimensões do dataset: {df2.shape}")
print(f"Colunas disponíveis: {list(df2.columns)}")

metrics2 = train_and_evaluate(
    df2, "MinTemp", "MaxTemp", "MinTemp vs MaxTemp", 
    save_path=os.path.join(RESULTS_PATH, "min_vs_max.png")
)

### 4.3 Dataset 3: Salinity vs Temperatura

In [ ]:
print("=== DATASET 3: Salinity vs Temperatura ===")
df3 = load_dataset(
    os.path.join(DATA_PATH, "ds_salinity_vs_temp_raw.csv"), 
    conversions=[("T_degC", None)]
)
print(f"Dimensões do dataset: {df3.shape}")
print(f"Colunas disponíveis: {list(df3.columns)}")

metrics3 = train_and_evaluate(df3, "Salnty", "T_degC", "Salinity vs Temperatura")

### 4.4 Salvamento das Métricas

In [ ]:
with open(os.path.join("results", "metrics.txt"), "w") as f:
    f.write("Resultados dos Modelos:\n")
    f.write(f"Dataset 1 (Umidade vs Temperatura): {metrics1}\n")
    f.write(f"Dataset 2 (MinTemp vs MaxTemp): {metrics2}\n")
    f.write(f"Dataset 3 (Salinity vs Temperatura): {metrics3}\n")

print("Métricas salvas em 'results/metrics.txt'")
print("\n=== RESUMO FINAL ===")
print(f"Dataset 1 - MAE: {metrics1['MAE']:.3f}, MSE: {metrics1['MSE']:.3f}, R²: {metrics1['R2']:.3f}")
print(f"Dataset 2 - MAE: {metrics2['MAE']:.3f}, MSE: {metrics2['MSE']:.3f}, R²: {metrics2['R2']:.3f}")
print(f"Dataset 3 - MAE: {metrics3['MAE']:.3f}, MSE: {metrics3['MSE']:.3f}, R²: {metrics3['R2']:.3f}")

## 5. Análise Comparativa dos Resultados

### 5.1 Tabela Resumo das Métricas

| Dataset | MAE | MSE | R² | Desempenho Relativo |
|---------|-----|-----|----|---------------------|
| Umidade vs Temperatura | | | | |
| MinTemp vs MaxTemp | | | | |
| Salinity vs Temperatura | | | | |

### 5.2 Discussão dos Resultados

**Dataset 2 (MinTemp vs MaxTemp):**
- Espera-se o melhor desempenho devido à forte relação física conhecida
- Temperaturas mínima e máxima têm correlação naturalmente alta

**Dataset 1 (Umidade vs Temperatura):**
- Relação mais complexa e influenciada por múltiplos fatores
- Possível comportamento não-linear em extremos de temperatura

**Dataset 3 (Salinity vs Temperatura):**
- Dinâmica oceanográfica complexa
- Pode exigir transformações ou modelos mais sofisticados

## 6. Conclusões

### 6.1 Principais Conclusões

1. **Eficácia do Modelo**: A Regressão Linear mostrou-se adequada para relações lineares bem definidas, como entre temperaturas mínima e máxima.

2. **Importância do Domínio**: O desempenho varia significativamente conforme a natureza física da relação entre as variáveis.

3. **Limitações Identificadas**: Para relações não-lineares, modelos mais complexos podem ser necessários.

### 6.2 Contribuições do Trabalho

- Implementação de pipeline reprodutível para análise comparativa
- Demonstração prática da importância da análise exploratória preliminar
- Base sólida para experimentos futuros com modelos mais avançados

### 6.3 Trabalhos Futuros

- Experimentar transformações de features (polinomiais, logarítmicas)
- Testar outros algoritmos (Random Forest, SVM, Redes Neurais)
- Incorporar mais variáveis explicativas
- Implementar validação cruzada para estimativas mais robustas